In [1]:
import pandas as pd
import requests
import urllib

### Downloading the Excel file from the link

In [4]:
urllib.request.urlretrieve("https://www.sec.gov/Archives/edgar/data/789019/000156459021002316/Financial_Report.xlsx", "FinancialReport.xlsx"); 

In [5]:
df=pd.read_excel("FinancialReport.xlsx",'BALANCE SHEETS')

In [6]:
start_asset=df[df['BALANCE SHEETS - USD ($) $ in Millions'].str.contains('Current assets',case=False)].index[0]
start_liabilities=df[df['BALANCE SHEETS - USD ($) $ in Millions'].str.contains('Current liabilities',case=False)].index[0]
start_stock=df[df['BALANCE SHEETS - USD ($) $ in Millions'].str.contains('Stockholders’ equity',case=False)].index[0]

### Getting all the rows of Current Liabilities :

In [7]:
df.iloc[start_liabilities+1:start_stock]

,BALANCE SHEETS - USD ($) $ in Millions,"Dec. 31, 2020","Jun. 30, 2020"
16,Accounts payable,12770,12530
17,Current portion of long-term debt,5387,3749
18,Accrued compensation,6838,7874
19,Short-term income taxes,1562,2130
20,Short-term unearned revenue,30402,36000
21,Other current liabilities,10527,10027
22,Total current liabilities,67486,72310
23,Long-term debt,55136,59578
24,Long-term income taxes,26701,29432
25,Long-term unearned revenue,2985,3180


### What proportion of “Total Current Liabilities” is “Accounts Payable”?

In [8]:
df_current_liabilities=df.iloc[start_liabilities+1:start_stock].set_index('BALANCE SHEETS - USD ($) $ in Millions').T

print(f"Proportion of Accounts Payable to Total Current Liabilities for {df_current_liabilities.index[0]} is {round((df_current_liabilities['Accounts payable']/df_current_liabilities['Total liabilities'])[0],3)}")
print(f"Proportion of Accounts Payable to Total Current Liabilities for {df_current_liabilities.index[1]} is {round((df_current_liabilities['Accounts payable']/df_current_liabilities['Total liabilities'])[1],3)}")

Proportion of Accounts Payable to Total Current Liabilities for Dec. 31, 2020 is 0.073
Proportion of Accounts Payable to Total Current Liabilities for Jun. 30, 2020 is 0.068


### How much percent is “Total Liabilities and stockholders equity” for both Dec, 31st 2020 more or less than Jun, 30th 2020?

In [9]:
df_t=df.set_index('BALANCE SHEETS - USD ($) $ in Millions').T
sum_dec31=df_t.iloc[0]['Total liabilities']+df_t.iloc[0]['Total stockholders’ equity']
sum_jun30=df_t.iloc[1]['Total liabilities']+df_t.iloc[1]['Total stockholders’ equity']
print(f"Percentage : {(sum_dec31-sum_jun30)/sum_jun30*100}")

Percentage : 0.937901371008692


### Combine all the sheets with the same column names (Dec, 31st 2020 and June 30th 2020) and create a master excel sheet with these two columns. The final master excel sheet should have four columns: the two dates mentioned above, variable name (like Cash and Cash Equivalents) and variable type (like Current Assets).

In [10]:
df2 = pd.concat(pd.read_excel('Financial_Report.xlsx', sheet_name=None), ignore_index=True)

In [17]:
df4 = df2.drop(['Dec. 31, 2020','Jun. 30, 2020'],axis=1)

In [18]:
df2['variablename'] = df4.apply(lambda row: ' or '.join(row.dropna().astype(str)), axis=1)

In [19]:
df_final = df2[['Dec. 31, 2020','Jun. 30, 2020','variablename']]

In [20]:
start_asset=df_final[df_final['variablename'].str.contains('Current assets',case=False)].index[0]
start_liabilities=df_final[df_final['variablename'].str.contains('Current liabilities',case=False)].index[0]
start_stock=df_final[df_final['variablename'].str.contains('Stockholders’ equity',case=False)].index[0]

In [21]:
df_new=df_final.copy()
df_new['Type']='NONE'
df_new.loc[start_asset+1:start_liabilities,'Type']='Current Assets'
df_new.loc[start_liabilities+1:start_stock,'Type']='Current Liabilities'
df_new.loc[start_stock+1:,'Type']="Stockholder's Equity"
df_new.dropna(inplace=True)

In [22]:
df_new

,"Dec. 31, 2020","Jun. 30, 2020",variablename,Type
83,14432,13576,Cash and cash equivalents,Current Assets
84,117536,122951,Short-term investments,Current Assets
85,131968,136527,"Total cash, cash equivalents, and short-term i...",Current Assets
86,27312,32011,"Accounts receivable, net of allowance for doub...",Current Assets
87,1924,1895,Inventories,Current Assets
...,...,...,...,...
1067,699,540,Total finance lease liabilities,Stockholder's Equity
1070,8875,7671,Total operating lease liabilities,Stockholder's Equity
1073,10819,8956,Total finance lease liabilities,Stockholder's Equity
1083,10741.0,9287.0,Operating Leases,Stockholder's Equity


In [23]:
df_new.to_excel("MasterSheet.xlsx",index=False)